# Analyzing Neural Time Series Data: Data extraction from MATLAB matrix

#### Andrew J. Graves on 12/03/19

### Get the data

In [ ]:
# Import modules
from scipy import io
import numpy as np
import mne

# Load Sample MATLAB/EEGLAB object from current directory
EEG = io.loadmat('sampleEEGdata.mat')['EEG']

# Get the data and reshape data from 3D (chan x time x trial) to 2D (chan x (time x trial))
three_d = EEG['data'].item()
two_d = three_d.reshape(three_d.shape[0], (three_d.shape[1]*three_d.shape[2]))

# Extract sampling rate, channel names, channel types, and time
samp_rate = EEG['srate'].item()
chan_names = sum(np.concatenate(EEG['chanlocs'].item()['labels'].tolist()).tolist(), [])
chan_types = ['eeg'] * len(chan_names)
eeg_time = EEG['times'].item()[0]

# Create an MNE info object and combine 2D data with info for an MNE raw object
info = mne.create_info(ch_names=chan_names, sfreq=samp_rate, ch_types=chan_types)
raw = mne.io.RawArray(two_d, info, verbose=False)

### Get useful functions

In [ ]:
# ERP
def get_erp(three_d, chan_index):
    return np.squeeze(np.mean(three_d[chan_index, :, :], 2))

# Low-pass filter for ERPs
def low_pass_erp(erp, filter_cutoff, trans_width, nyq):
    f_freqs = np.array([0, filter_cutoff, filter_cutoff * (1 + trans_width), nyq]) / nyq
    ideal_resp = np.array([1, 1, 0, 0])
    filter_weights = sp.firls(101, f_freqs, ideal_resp)
    return sp.filtfilt(filter_weights, 1, erp)

# Band-pass filter for ERPs
def band_pass_erp(erp, filter_low, filter_high, trans_width, nyq):
    f_freqs = np.array([0, filter_low * (1 - trans_width), filter_low, 
                        filter_high, filter_high * (1 + trans_width), nyq]) / nyq
    ideal_resp = np.array([0, 0, 1, 1, 0, 0])
    filter_weights = sp.firls(round(3 * (samp_rate.item() / filter_low) + 1), f_freqs, ideal_resp)
    return sp.filtfilt(filter_weights, 1, erp)